# IMPORT PACKAGES

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix ,roc_curve, auc
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns

# CLEANING DATA

In [2]:
df= pd.read_csv("PhiUSIIL_Phishing_URL_Dataset.csv")

df.head()

,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,521848.txt,https://www.southbankmosaics.com,31,www.southbankmosaics.com,24,0,com,100.0,1.000000,0.522907,...,0,0,1,34,20,28,119,0,124,1
1,31372.txt,https://www.uni-mainz.de,23,www.uni-mainz.de,16,0,de,100.0,0.666667,0.032650,...,0,0,1,50,9,8,39,0,217,1
2,597387.txt,https://www.voicefmradio.co.uk,29,www.voicefmradio.co.uk,22,0,uk,100.0,0.866667,0.028555,...,0,0,1,10,2,7,42,2,5,1
3,554095.txt,https://www.sfnmjournal.com,26,www.sfnmjournal.com,19,0,com,100.0,1.000000,0.522907,...,1,1,1,3,27,15,22,1,31,1
4,151578.txt,https://www.rewildingargentina.org,33,www.rewildingargentina.org,26,0,org,100.0,1.000000,0.079963,...,1,0,1,244,15,34,72,1,85,1


In [3]:
df.columns

Index(['FILENAME', 'URL', 'URLLength', 'Domain', 'DomainLength', 'IsDomainIP',
       'TLD', 'URLSimilarityIndex', 'CharContinuationRate',
       'TLDLegitimateProb', 'URLCharProb', 'TLDLength', 'NoOfSubDomain',
       'HasObfuscation', 'NoOfObfuscatedChar', 'ObfuscationRatio',
       'NoOfLettersInURL', 'LetterRatioInURL', 'NoOfDegitsInURL',
       'DegitRatioInURL', 'NoOfEqualsInURL', 'NoOfQMarkInURL',
       'NoOfAmpersandInURL', 'NoOfOtherSpecialCharsInURL',
       'SpacialCharRatioInURL', 'IsHTTPS', 'LineOfCode', 'LargestLineLength',
       'HasTitle', 'Title', 'DomainTitleMatchScore', 'URLTitleMatchScore',
       'HasFavicon', 'Robots', 'IsResponsive', 'NoOfURLRedirect',
       'NoOfSelfRedirect', 'HasDescription', 'NoOfPopup', 'NoOfiFrame',
       'HasExternalFormSubmit', 'HasSocialNet', 'HasSubmitButton',
       'HasHiddenFields', 'HasPasswordField', 'Bank', 'Pay', 'Crypto',
       'HasCopyrightInfo', 'NoOfImage', 'NoOfCSS', 'NoOfJS', 'NoOfSelfRef',
       'NoOfEmptyRef', 'NoOf

In [4]:
#profile = ProfileReport(df, title="phishin")
#profile.to_notebook_iframe()

In [5]:
#profile.to_file(output_file="REPORT.html")
# Generate profile report
profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)
profile.to_file(output_file="REPORT.html")
# Display the report as widgets
profile.to_widgets()

print("Profiling Report displayed as widgets.")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\Latif\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ydata_profiling\model\typeset.py:125: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  not pdt.is_categorical_dtype(series)
C:\Users\Latif\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ydata_profiling\model\typeset.py:125: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  not pdt.is_categorical_dtype(series)
C:\Users\Latif\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ydata_profiling\model\typeset_relations.py:34: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstanc

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Profiling Report displayed as widgets.


In [6]:
# Get the list of categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# Print the list of categorical columns
print("Categorical Columns:")
for col in categorical_cols:
    print(col)

Categorical Columns:
FILENAME
URL
Domain
TLD
Title


In [7]:
df= df.drop(columns=['FILENAME', 'URL', 'Domain', 'Title'])

df = pd.get_dummies(df, columns=['TLD'])
df.head()

,URLLength,DomainLength,IsDomainIP,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,NoOfSubDomain,HasObfuscation,...,TLD_xyz,TLD_yachts,TLD_ye,TLD_yoga,TLD_youtube,TLD_yt,TLD_za,TLD_zm,TLD_zone,TLD_zw
0,31,24,0,100.0,1.000000,0.522907,0.061933,3,1,0,...,False,False,False,False,False,False,False,False,False,False
1,23,16,0,100.0,0.666667,0.032650,0.050207,2,1,0,...,False,False,False,False,False,False,False,False,False,False
2,29,22,0,100.0,0.866667,0.028555,0.064129,2,2,0,...,False,False,False,False,False,False,False,False,False,False
3,26,19,0,100.0,1.000000,0.522907,0.057606,3,1,0,...,False,False,False,False,False,False,False,False,False,False
4,33,26,0,100.0,1.000000,0.079963,0.059441,3,1,0,...,False,False,False,False,False,False,False,False,False,False


In [8]:
X = df.drop(columns=['label'])
y = df['label']

In [9]:
X.head()

,URLLength,DomainLength,IsDomainIP,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,NoOfSubDomain,HasObfuscation,...,TLD_xyz,TLD_yachts,TLD_ye,TLD_yoga,TLD_youtube,TLD_yt,TLD_za,TLD_zm,TLD_zone,TLD_zw
0,31,24,0,100.0,1.000000,0.522907,0.061933,3,1,0,...,False,False,False,False,False,False,False,False,False,False
1,23,16,0,100.0,0.666667,0.032650,0.050207,2,1,0,...,False,False,False,False,False,False,False,False,False,False
2,29,22,0,100.0,0.866667,0.028555,0.064129,2,2,0,...,False,False,False,False,False,False,False,False,False,False
3,26,19,0,100.0,1.000000,0.522907,0.057606,3,1,0,...,False,False,False,False,False,False,False,False,False,False
4,33,26,0,100.0,1.000000,0.079963,0.059441,3,1,0,...,False,False,False,False,False,False,False,False,False,False


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Creating and training the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Making predictions on the testing set
y_pred = rf_classifier.predict(X_test)

# Calculating the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9999575902796921


In [12]:
# Generate a classification report
print("Classification Report:")
print(classification_report(y_test, y_pred,digits=20))

Classification Report:
                      precision    recall  f1-score   support

                   0  1.00000000000000000000 0.99990061617968595264 0.99995030562043429878     20124
                   1  0.99992602729592783284 1.00000000000000000000 0.99996301227992312466     27035

            accuracy                      0.99995759027969211008     47159
           macro avg  0.99996301364796391642 0.99995030808984297632 0.99995665895017871172     47159
        weighted avg  0.99995759341685375166 0.99995759027969211008 0.99995759001024919765     47159



In [13]:
# Generate a confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Confusion Matrix:
[[20122     2]
 [    0 27035]]


In [14]:
# Plotting the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, cmap='Blues', fmt='d', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Plotting feature importances
plt.figure(figsize=(10, 6))
feat_importances = pd.Series(rf_classifier.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')
plt.title('Top 15 Most Important Features')
plt.xlabel('Relative Importance')
plt.ylabel('Features')
plt.show()

# Plotting ROC curve

probs = rf_classifier.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, probs[:,1])
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


C:\Users\Latif\AppData\Local\Temp\ipykernel_22696\3474202667.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\Latif\AppData\Local\Temp\ipykernel_22696\3474202667.py:16: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\Latif\AppData\Local\Temp\ipykernel_22696\3474202667.py:31: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
